## Part I

Consider a moving set of column indices

In [31]:
import numpy as np

with open('input/7-input.txt', 'r') as f:
    lines = f.read().splitlines()

grid = np.array([list(line) for line in lines]) == "^"

In [ ]:
def advance_beam(grid,distance,beam_set):
    if distance < len(grid):
        current_indices = np.array(list(beam_set))
        # Filter for valid column indices
        valid_mask = (current_indices >= 0) & (current_indices < grid.shape[1])
        valid_indices = current_indices[valid_mask]
        
        t = valid_indices[grid[distance, valid_indices]]
        
        beam_set.difference_update(set(t))
        beam_set.update(set(np.unique(np.concatenate([t-1, t+1])).tolist()))
        
        return len(t) + advance_beam(grid,distance+1,beam_set)
    else:
        return 0

In [121]:
result = advance_beam(grid, distance = 0, beam_set = {70})

In [122]:
result

(1550, np.int64(9897897326778))

## Part II

In [123]:
from collections import defaultdict

def advance_beam(grid, distance, beam_set):
    # Initialize beam_counts if it's a set (first call)
    if isinstance(beam_set, set):
        beam_counts = defaultdict(int, {x: 1 for x in beam_set})
    else:
        beam_counts = beam_set

    if distance < len(grid):
        current_indices = np.array(list(beam_counts.keys()))
        current_counts = np.array(list(beam_counts.values()))
        
        # Filter for valid column indices
        valid_mask = (current_indices >= 0) & (current_indices < grid.shape[1])
        valid_indices = current_indices[valid_mask]
        valid_counts = current_counts[valid_mask]
        
        # Identify splitters
        is_splitter = grid[distance, valid_indices]
        
        # Create new state
        new_counts = defaultdict(int)
        
        # Update counts for next row
        for col, count, is_split in zip(valid_indices, valid_counts, is_splitter):
            if is_split:
                new_counts[col-1] += count
                new_counts[col+1] += count
            else:
                new_counts[col] += count
            
        splits, routes = advance_beam(grid, distance + 1, new_counts)
        return np.sum(is_splitter) + splits, routes
    else:
        return 0, sum(beam_counts.values())